This script imputes Veteran's Benefits (VB) recipients dollar benefit amount to match the aggregates with United States Department of Veterans Affairs (USVA) statistics for VB. In this current version, we used 2015 CPS data and USVA FY2014 and FY2015 annual reports on VB. Please refer to the documentation in the same folder for more details on methodology and assumptions. The output this script is an individual level dataset that contains CPS personal level ID (PERIDNUM), individual participation indicator (vb_participation, 0 - not a recipient, 1 - current recipient on file, 2 - imputed recipient), and benefit amount.

Input: 2015 CPS (cpsmar2015t.csv), number of recipients and their benefits amount by state in 2014 (VB_administrative.csv)

Output: VB_Imputation.csv

Additional Source links: https://www.va.gov/vetdata/ 

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import random
import statsmodels.discrete.discrete_model as sm
import matplotlib.pyplot as plt 

##### Variables we use in Veteran's Benefits

In [2]:
CPS_dataset = pd.read_csv('asec2015_pubuse.csv')
columns_to_keep = ['hvet_yn','hvetval','fvetval','finc_vet','sur_yn','srvs_val','sur_sc1','sur_val1','tsurval1','sur_sc2',
                   'sur_val2','tsurval2','vet_val','vet_yn','vet_typ1','vet_typ2','vet_typ3','vet_typ4','vet_typ5','hsur_yn',
                   'hsurval','fsurval','finc_sur','gestfips','marsupwt','peafever','champ','a_age','wsal_val','semp_val','frse_val',
                   'pedisdrs', 'pedisear', 'pediseye', 'pedisout', 'pedisphy', 'pedisrem','a_sex','peridnum','fh_seq','wc_yn', 'ss_yn', 
                   'dis_yn', 'hed_yn', 'hcsp_yn', 'hfdval','paw_yn', 'uc_yn', 'mcaid','mcare']
CPS_dataset = CPS_dataset[columns_to_keep]
CPS_dataset.to_csv('VB.csv', columns=columns_to_keep, index=False)

C:\Users\comm6\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (5,22,23,28,80,187,273) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
CPS_dataset = pd.read_csv('VB.csv')

In [4]:
# VB value
vbvalue = pd.to_numeric(np.where(CPS_dataset.vet_val!= 'None or not in universe', CPS_dataset.vet_val, 0))
# VB indicator
indicator = pd.to_numeric(np.where(CPS_dataset.vet_yn== 'Yes', 1, 0))
# On duty before
active = pd.to_numeric(np.where(CPS_dataset.peafever== 'Yes', 1, 0))

In [5]:
VB = DataFrame(vbvalue.transpose())
VB.columns = ['vbvalue']

In [6]:
# 5 types of VB
VB['kind'] = pd.to_numeric(np.where(CPS_dataset.vet_typ1== 'Yes', 1, 0))
VB.kind = pd.to_numeric(np.where(CPS_dataset.vet_typ2== 'Yes', 2, VB.kind))
VB.kind = pd.to_numeric(np.where(CPS_dataset.vet_typ3== 'Yes', 3, VB.kind))
VB.kind = pd.to_numeric(np.where(CPS_dataset.vet_typ4== 'Yes', 4, VB.kind))
VB.kind = pd.to_numeric(np.where(CPS_dataset.vet_typ5== 'Yes', 5, VB.kind))

In [7]:
VB['indicator'] = indicator
VB['active'] = active
VB['marsupwt'] = CPS_dataset.marsupwt
VB['fh_seq'] = CPS_dataset.fh_seq
VB['gestfips'] = CPS_dataset.gestfips
VB['peridnum'] = CPS_dataset.peridnum

In [8]:
# Prepare income information
wage = pd.to_numeric(np.where(CPS_dataset.wsal_val!= 'None or not in universe', CPS_dataset.wsal_val, 0))
self_employed1 = pd.to_numeric(np.where(CPS_dataset.semp_val!= 'None or not in universe', CPS_dataset.semp_val, 0))
self_employed2 = pd.to_numeric(np.where(CPS_dataset.frse_val!= 'None or not in universe', CPS_dataset.frse_val, 0))
income = wage + self_employed1 + self_employed2
VB['income'] = income

In [9]:
# Prepare age information
CPS_dataset.a_age = np.where(CPS_dataset.a_age == "80-84 years of age",
                             random.randrange(80, 84),
                             CPS_dataset.a_age)
CPS_dataset.a_age = np.where(CPS_dataset.a_age == "85+ years of age",
                             random.randrange(85, 95),
                             CPS_dataset.a_age)
VB['a_age'] = pd.to_numeric(CPS_dataset.a_age)

In [10]:
# Prepare gender inforamtion, 0 for male, 1 for female
VB['sex'] = pd.to_numeric(np.where(CPS_dataset.a_sex == 'Male', 0, 1))

In [11]:
# Prepare disabilty information, 6 types
d1 = np.where(CPS_dataset.pedisdrs == 'Yes',1,0)
d2 = np.where(CPS_dataset.pedisear == 'Yes',1,0)
d3 = np.where(CPS_dataset.pediseye == 'Yes',1,0)
d4 = np.where(CPS_dataset.pedisout == 'Yes',1,0)
d5 = np.where(CPS_dataset.pedisphy == 'Yes',1,0)
d6 = np.where(CPS_dataset.pedisrem == 'Yes',1,0)
VB['d1'] = d1
VB['d2'] = d2
VB['d3'] = d3
VB['d4'] = d4
VB['d5'] = d5
VB['d6'] = d6

In [12]:
# Limitation of potential recipients
familyactive = VB.groupby('fh_seq')['active'].sum()
factive = DataFrame(familyactive)

In [13]:
# Family active index
VB['familyactive'] = np.zeros(len(VB))
for number in VB.fh_seq:
    VB.familyactive= np.where(VB.fh_seq==number,factive.active[factive.index==number],VB.familyactive)

Add dummy for program participation 

In [14]:
VB['paw_yn'] = np.where(CPS_dataset.paw_yn=='Yes', 1, 0)
VB['wc_yn'] = np.where(CPS_dataset.wc_yn=='Yes', 1, 0)
VB['uc_yn'] = np.where(CPS_dataset.uc_yn=='Yes', 1, 0)
VB['ss_yn'] = np.where(CPS_dataset.ss_yn=='Yes', 1, 0)
VB['sur_yn'] = np.where(CPS_dataset.sur_yn=='Yes', 1, 0)
VB['dis_yn'] = np.where(CPS_dataset.dis_yn=='Yes', 1, 0)
VB['hed_yn'] = np.where(CPS_dataset.hed_yn=='Yes', 1, 0)
VB['hcsp_yn'] = np.where(CPS_dataset.hcsp_yn=='Yes', 1, 0)
VB['hfdval'] = np.where(CPS_dataset.hfdval!='Not in universe', 1, 0)
VB['mcare'] = np.where(CPS_dataset.mcare=='Yes', 1, 0)
VB['mcaid'] = np.where(CPS_dataset.mcaid=='Yes', 1, 0)

## Regression model

In [15]:
dta = VB
dta['intercept'] = np.ones(len(dta))
model = sm.Logit(endog=dta.indicator, exog=dta[['a_age','sex','income','d1','d2','d3','d4','d5','d6',
                                                'active','paw_yn','wc_yn','ss_yn', 'uc_yn', 'sur_yn',
                                                'hed_yn','hcsp_yn', 'hfdval','mcaid','mcare','intercept']]).fit()

Optimization terminated successfully.
         Current function value: 0.040644
         Iterations 10


In [16]:
dta.marsupwt[(dta.indicator==1)].sum()

3650667.969999996

In [17]:
#print model.summary()
print (model.summary()) #add parentheses so that print function would work with Python 3.6 version

                           Logit Regression Results                           
Dep. Variable:              indicator   No. Observations:               199024
Model:                          Logit   Df Residuals:                   199003
Method:                           MLE   Df Model:                           20
Date:                Fri, 04 Aug 2017   Pseudo R-squ.:                  0.3376
Time:                        09:10:16   Log-Likelihood:                -8089.1
converged:                       True   LL-Null:                       -12211.
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
a_age          0.0062      0.002      3.494      0.000       0.003       0.010
sex           -0.3474      0.070     -4.987      0.000      -0.484      -0.211
income      1.184e-07   4.56e-07      0.260      0.7

In [18]:
probs = model.predict()

## Import administrative data

In [19]:
admin = pd.read_csv('VB_administrative.csv',
                    dtype={ 'Total VB population': np.float,'Average benefits': np.float, 'Total benefits': np.float, 'Medical care': np.float})
admin.index = admin.Fips

In [20]:
# CPS total benefits and Administrative total benefits
state_benefit = {}
state_recipients = {}
for state in admin.Fips:
    this_state = (VB.gestfips==state)
    CPS_totalb = (VB.vbvalue * VB.marsupwt)[this_state].sum()
    admin_totalb =  admin['Total benefits'][state] 
    CPS_totaln = VB.marsupwt[this_state&VB.indicator==1].sum()
    admin_totaln =  admin["Total veteran receving benefits"][state]

    temp = [admin.State[state], CPS_totalb, admin_totalb, CPS_totaln, admin_totaln]
    state_benefit[state] = temp
    
pre_augment_benefit = DataFrame(state_benefit).transpose()
pre_augment_benefit.columns = ['State', 'CPS total benefits','Admin total benefits',
                               'CPS total individual recipients','Admin total individual recipients']

In [21]:
#pre_augment_benefit.to_csv('C:\Users\wangy\OneDrive\Documents\BasicIncomeProject\VB-pre-blow-up.csv')

## Imputation

In [22]:
# caculate difference of SNAP stats and CPS aggregates on recipients number
# by state
diff = {'Fips':[],'Difference in Population':[],'Mean Benefit':[],'CPS Population':[],'VA Population':[]}
diff['Fips'] = admin.Fips
current = (VB.indicator==1)
for FIPS in admin.Fips:
        this_state = (VB.gestfips==FIPS)
        current_tots = VB.marsupwt[current&this_state].sum()
        valid_num = VB.marsupwt[current&this_state].sum() + 0.0000001
        current_mean = ((VB.vbvalue * VB.marsupwt)[current&this_state].sum())/valid_num
        diff['CPS Population'].append(current_tots)
        diff['VA Population'].append(float(admin["Total veteran receving benefits"][admin.Fips == FIPS]))
        diff['Difference in Population'].append(float(admin["Total veteran receving benefits"][admin.Fips == FIPS])- current_tots)
        diff['Mean Benefit'].append(current_mean)

In [23]:
d = DataFrame(diff)
d = d[['Fips', 'Mean Benefit', 'Difference in Population', 'CPS Population', 'VA Population']]
#d.to_csv('recipients.csv', index=False)

In [24]:
VB['impute'] = np.zeros(len(VB))
VB['vb_impute'] = np.zeros(len(VB))

non_current = (VB.indicator==0)
current = (VB.indicator==1)
random.seed()

for FIPS in admin.Fips:
    
        print ('we need to impute', d['Difference in Population'][FIPS], 'for state', FIPS)
        
        if d['Difference in Population'][FIPS] < 0:
            continue
        else:
            this_state = (VB.gestfips==FIPS)
            not_imputed = (VB.impute==0)
            pool_index = VB[this_state&not_imputed&non_current].index
            pool = DataFrame({'weight': VB.marsupwt[pool_index], 'prob': probs[pool_index]},
                            index=pool_index)
            pool = pool.sort_values(by='prob', ascending=False)
            pool['cumsum_weight'] = pool['weight'].cumsum()
            pool['distance'] = abs(pool.cumsum_weight-d['Difference in Population'][FIPS])
            min_index = pool.sort_values(by='distance')[:1].index
            min_weight = int(pool.loc[min_index].cumsum_weight)
            pool['impute'] = np.where(pool.cumsum_weight<=min_weight+10 , 1, 0)
            VB.impute[pool.index[pool['impute']==1]] = 1
            VB.vb_impute[pool.index[pool['impute']==1]] = admin['Average benefits'][FIPS]

        print ('Method1: regression gives', 
                VB.marsupwt[(VB.impute==1)&this_state].sum()) 

we need to impute 63983.93 for state 1
Method1: regression gives 62555.590000000004
we need to impute 7996.75 for state 2
Method1: regression gives 8110.029999999998
we need to impute 19387.72 for state 4


C:\Users\comm6\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\comm6\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Method1: regression gives 18227.89
we need to impute 10768.3 for state 5
Method1: regression gives 10186.400000000001
we need to impute 102333.73 for state 6
Method1: regression gives 102114.15999999997
we need to impute 11727.7 for state 8
Method1: regression gives 11748.65
we need to impute -3928.16 for state 9
we need to impute 1121.21 for state 10
Method1: regression gives 967.34
we need to impute 1873.39 for state 11
Method1: regression gives 1800.39
we need to impute 137895.24 for state 12
Method1: regression gives 137986.86000000002
we need to impute 59480.11 for state 13
Method1: regression gives 60370.05999999999
we need to impute 7654.82 for state 15
Method1: regression gives 7759.27
we need to impute 4395.78 for state 16
Method1: regression gives 4318.64
we need to impute -1740.29 for state 17
we need to impute 20139.84 for state 18
Method1: regression gives 21805.78
we need to impute 18910.13 for state 19
Method1: regression gives 18066.74
we need to impute 8579.79 for stat

In [25]:
#Adjustment ratio
results = {}

imputed = (VB.impute == 1)
has_val = (VB.vbvalue != 0)
no_val = (VB.vbvalue == 0)

for FIPS in admin.Fips:
    this_state = (VB.gestfips==FIPS)
    
    current_total = (VB.vbvalue * VB.marsupwt)[this_state].sum() 
    imputed_total = (VB.vb_impute * VB.marsupwt)[this_state&imputed].sum()
    on_file = current_total + imputed_total

    admin_total = admin['Total benefits'][FIPS]
    
    adjust_ratio = admin_total / on_file
    this_state_num = [admin['State'][FIPS], on_file, admin_total, adjust_ratio]
    results[FIPS] = this_state_num
    

    VB.vb_impute = np.where(has_val&this_state, VB.vbvalue * adjust_ratio, VB.vb_impute)
    VB.vb_impute = np.where(no_val&this_state, VB.vb_impute * adjust_ratio, VB.vb_impute)

VB["vb_participation"] = np.zeros(len(VB))
VB["vb_participation"] = np.where(VB.impute==1, 2, 0)#Augmented
VB["vb_participation"] = np.where(has_val, 1, VB.vb_participation)#CPS 


r = DataFrame(results).transpose()
r.columns=['State', 'Imputed', 'Admin', 'adjust ratio']
#r.to_csv('amount.csv', index=False)

## Assign Medical care benefit

In [26]:
#Medical care
medical = {}
medicalcare = {}
for FIPS in admin.Fips:
    this_state = (VB.gestfips==FIPS)
    medical[FIPS] = admin['Medical care'][FIPS] / (VB.marsupwt[VB.vb_participation==1][this_state].sum())
    VB.vb_impute = np.where((VB.vb_participation==1) & (this_state), VB.vb_impute + medical[FIPS], VB.vb_impute)
    medicalcare[FIPS] = [admin['State'][FIPS], medical[FIPS]]

In [27]:
VB.to_csv('VB_Imputation.csv', 
         columns=['peridnum','vb_participation', 'vb_impute'],
         index = False)

In [28]:
r = DataFrame(medicalcare).transpose()
r.columns=['State','Individual average medical care']
r.to_csv('medical.csv', index=False)